In [ ]:
from pathlib import Path
from dsa110_contimg.mosaic.orchestrator import MosaicOrchestrator
from dsa110_contimg.conversion.transit_precalc import precalculate_transits_for_calibrator
from dsa110_contimg.database.products import ensure_products_db
from dsa110_contimg.photometry.manager import PhotometryConfig
import sqlite3

# CRITICAL: Set PIPELINE_STATE_DIR to point to the state directory (not a specific database file)
# This is the directory containing all state files: products.sqlite3, hdf5.sqlite3, catalogs/, etc.
import os
os.environ["PIPELINE_STATE_DIR"] = "/data/dsa110-contimg/state"

# CRITICAL: We MUST use vla_calibrators.sqlite3
# The catalog loading function has been fixed to properly index by calibrator name
state_dir = Path(os.getenv("PIPELINE_STATE_DIR", "/data/dsa110-contimg/state"))
vla_catalog_path = state_dir / "catalogs" / "vla_calibrators.sqlite3"

print(f"PIPELINE_STATE_DIR: {os.getenv('PIPELINE_STATE_DIR')}")
print(f"Using VLA catalog: {vla_catalog_path}")
print(f"Catalog exists: {vla_catalog_path.exists()}")

# Ensure transit times are pre-calculated (enables fast querying from database)
# Transit pre-calculation stores transit times in calibrator_transits table for fast lookup
print("\nChecking transit pre-calculation...")
products_db_path = state_dir / "products.sqlite3"
products_db = ensure_products_db(products_db_path)

# Get calibrator coordinates
catalog_db = state_dir / "catalogs" / "vla_calibrators.sqlite3"
cat_conn = sqlite3.connect(str(catalog_db))
cursor = cat_conn.cursor()
cursor.execute("SELECT ra_deg, dec_deg FROM calibrators WHERE name = '0834+555'")
cal = cursor.fetchone()
cat_conn.close()

if cal:
    ra_deg, dec_deg = cal
    # Check if transits already pre-calculated
    cursor = products_db.cursor()
    cursor.execute("""
        SELECT COUNT(*) FROM calibrator_transits 
        WHERE calibrator_name = '0834+555' AND has_data = 1
    """)
    existing_count = cursor.fetchone()[0]
    
    if existing_count == 0:
        print(f"Pre-calculating transit times for 0834+555...")
        transits_with_data = precalculate_transits_for_calibrator(
            products_db=products_db,
            calibrator_name="0834+555",
            ra_deg=ra_deg,
            dec_deg=dec_deg,
            max_days_back=60,
        )
        print(f":check_mark: Pre-calculated {transits_with_data} transits with available data")
    else:
        print(f":check_mark: Transit times already pre-calculated ({existing_count} transits with data)")
else:
    print("ERROR: Calibrator 0834+555 not found in catalog")

# Initialize orchestrator with photometry enabled (default as of our latest changes)
# Photometry configuration: uses NVSS catalog, 1.0° radius for mosaics, 10 mJy threshold
print("\n" + "="*60)
print("Initializing MosaicOrchestrator")
print("="*60)

photometry_config = {
    "catalog": "nvss",
    "radius_deg": 1.0,
    "min_flux_mjy": 10.0,
    "method": "peak",  # Fast peak photometry (default)
    "normalize": False,
    "detect_ese": False,
}

orchestrator = MosaicOrchestrator(
    enable_photometry=True,  # :check_mark: Now default, but explicit for clarity
    photometry_config=photometry_config,
)

print(f":check_mark: Photometry enabled: {orchestrator.enable_photometry}")
if orchestrator.photometry_manager:
    print(f":check_mark: PhotometryManager initialized")
    print(f"  - Catalog: {orchestrator.photometry_manager.default_config.catalog}")
    print(f"  - Method: {orchestrator.photometry_manager.default_config.method}")
    print(f"  - Min flux: {orchestrator.photometry_manager.default_config.min_flux_mjy} mJy")

# Verify calibrator service is initialized with SQLite catalog
if orchestrator.calibrator_service:
    sqlite_catalogs = [c for c in orchestrator.calibrator_service.catalogs if str(c).endswith('.sqlite3')]
    if sqlite_catalogs:
        print(f":check_mark: Using SQLite catalog: {sqlite_catalogs[0]}")
    else:
        print(":warning_sign: WARNING: No SQLite catalog found!")
else:
    print("WARNING: Calibrator service not initialized")

print("\n" + "="*60)
print("Creating 3-tile mosaic (15 minutes) centered on 0834+555")
print("="*60)
print("\n⏱:variation_selector-16:  EXPECTED WORKFLOW:")
print("1. HDF5→MS conversion (if needed)")
print("2. MODEL_DATA seeding after calibration")
print("3. Calibration solve (BP + Gain)")
print("4. MODEL_DATA validation before imaging")
print("5. WSClean imaging")
print("6. Mosaic creation")
print("7. Photometry measurement (NVSS sources)")
print("8. Publish gating (waits for photometry_status='completed')")
print("")

mosaic_path = orchestrator.create_mosaic_centered_on_calibrator(
    calibrator_name="0834+555",
    timespan_minutes=15,  # 3 tiles × 5 minutes = 15 minutes
    wait_for_published=True,
    dry_run=False,
    overwrite=False,
)

print(f"\n{'='*60}")
if mosaic_path:
    print(f":check_mark: SUCCESS: Mosaic created at: {mosaic_path}")
    print(f"{'='*60}")
    
    # Verify photometry was completed
    import sqlite3
    registry_db = state_dir / "data_registry.sqlite3"
    if registry_db.exists():
        conn = sqlite3.connect(str(registry_db))
        cursor = conn.cursor()
        cursor.execute("""
            SELECT data_id, photometry_status, photometry_job_id 
            FROM data_registry 
            WHERE data_type='mosaic' 
            ORDER BY created_at DESC 
            LIMIT 1
        """)
        result = cursor.fetchone()
        conn.close()
        
        if result:
            data_id, phot_status, job_id = result
            print(f"\n:bar_chart: PHOTOMETRY STATUS:")
            print(f"  - Mosaic ID: {data_id}")
            print(f"  - Photometry status: {phot_status}")
            print(f"  - Batch job ID: {job_id}")
            
            # Count measurements
            products_conn = sqlite3.connect(str(products_db_path))
            cursor = products_conn.cursor()
            cursor.execute("""
                SELECT COUNT(*), AVG(peak_jyb), AVG(peak_err_jyb)
                FROM photometry
                WHERE image_path LIKE ?
            """, (f"%{Path(mosaic_path).name}%",))
            count, avg_flux, avg_err = cursor.fetchone()
            products_conn.close()
            
            if count and count > 0:
                print(f"  - Measurements: {count} sources")
                print(f"  - Avg flux: {avg_flux:.4f} Jy/beam")
                print(f"  - Avg error: {avg_err:.4f} Jy/beam")
            else:
                print(f"  :warning_sign::variation_selector-16:  No photometry measurements found")
else:
    print(f":ballot_x: FAILED: Mosaic creation returned None")
print(f"{'='*60}")

2025-11-17 19:04:57,403 - INFO - Creating 15-minute mosaic centered on 0834+555
2025-11-17 19:04:57,418 - INFO - Using default (earliest) transit: 2025-10-02T15:41:35.905
2025-11-17 19:04:57,431 - INFO - Transit time: 2025-10-02T15:41:35.905
Window: 2025-10-02T15:34:05.905 to 2025-10-02T15:49:05.905
Required MS files: 3
2025-11-17 19:04:57,433 - INFO - Only 2 MS files found, need 3. Triggering HDF5 conversion...


PIPELINE_STATE_DIR: /data/dsa110-contimg/state
Using VLA catalog: /data/dsa110-contimg/state/catalogs/vla_calibrators.sqlite3
Catalog exists: True

Checking transit pre-calculation...
:check_mark: Transit times already pre-calculated (13 transits with data)
:check_mark: Using SQLite catalog: /data/dsa110-contimg/state/catalogs/vla_calibrators.sqlite3

Creating 3-tile mosaic (15 minutes) centered on 0834+555


2025-11-17 19:05:03,420 - INFO - Triggering HDF5 conversion for window: 2025-10-02 15:34:05.905 to 2025-10-02 15:49:05.905
2025-11-17 19:05:05,471 - INFO - Found 3 complete subband group(s) (48 total files)
2025-11-17 19:05:05,472 - INFO -   Group 1:
2025-11-17 19:05:05,473 - INFO -     2025-10-02T15:34:57_sb00.hdf5
2025-11-17 19:05:05,474 - INFO -     2025-10-02T15:34:57_sb01.hdf5
2025-11-17 19:05:05,475 - INFO -     2025-10-02T15:34:56_sb02.hdf5
2025-11-17 19:05:05,475 - INFO -     2025-10-02T15:34:57_sb03.hdf5
2025-11-17 19:05:05,476 - INFO -     2025-10-02T15:34:57_sb04.hdf5
2025-11-17 19:05:05,477 - INFO -     2025-10-02T15:34:57_sb05.hdf5
2025-11-17 19:05:05,477 - INFO -     2025-10-02T15:34:57_sb06.hdf5
2025-11-17 19:05:05,478 - INFO -     2025-10-02T15:34:57_sb07.hdf5
2025-11-17 19:05:05,479 - INFO -     2025-10-02T15:34:57_sb08.hdf5
2025-11-17 19:05:05,480 - INFO -     2025-10-02T15:34:57_sb09.hdf5
2025-11-17 19:05:05,481 - INFO -     2025-10-02T15:34:56_sb10.hdf5
2025-11-17 1

Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms::SPECTRAL_WINDOW: 16 columns, 16 rows


/opt/miniforge/envs/casa6/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
2025-11-17 19:08:27,300 - INFO - Fixed FIELD table phase centers after concatenation


...

Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms: 26 columns, 1787904 rows


2025-11-18 03:26:16	SEVERE	msmetadata_cmpt.cc::timesforscans	Exception Reported: Exception: No matching scans found.
2025-11-18 03:26:16	SEVERE	msmetadata_cmpt.cc::timesforscans+	... thrown by std::set<casacore::ScanKey> casac::msmetadata::_getScanKeys(const std::vector<long int>&, long int, long int) const at File: /home/conda/feedstock_root/build_artifacts/casatools_1757431503580/work/casatools/src/tools/msmetadata/msmetadata_cmpt.cc, line: 2522


Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:40:06.ms: 26 columns, 1787904 rows


2025-11-18 03:26:18	SEVERE	msmetadata_cmpt.cc::timesforscans	Exception Reported: Exception: No matching scans found.
2025-11-18 03:26:18	SEVERE	msmetadata_cmpt.cc::timesforscans+	... thrown by std::set<casacore::ScanKey> casac::msmetadata::_getScanKeys(const std::vector<long int>&, long int, long int) const at File: /home/conda/feedstock_root/build_artifacts/casatools_1757431503580/work/casatools/src/tools/msmetadata/msmetadata_cmpt.cc, line: 2522
2025-11-17 19:26:17,832 - INFO - Found and validated calibrator 0834+555 in /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms at fields 0~1 (peak field: 0, flux: 2.500 Jy, PB response: 0.713)
2025-11-17 19:26:17,834 - INFO - Solving bandpass calibration using 0834+555 in /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms at fields 0~1
2025-11-17 19:26:17,835 - INFO - Validating MODEL_DATA for bandpass solve on field(s) 0~1...
2025-11-17 19:26:17,839 - ERROR - Bandpass calibration solving returned no tables
202

Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:45:15.ms: 26 columns, 1787904 rows
Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms::FIELD: 9 columns, 24 rows
Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms::FIELD: 9 columns, 24 rows
Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:34:57.ms: 26 columns, 1787904 rows
Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:40:06.ms::FIELD: 9 columns, 24 rows
Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:40:06.ms::FIELD: 9 columns, 24 rows
Successful readonly open of default-locked table /stage/dsa110-contimg/ms/science/2025-10-02/2025-10-02T15:40:06.ms::FIELD: 9 columns, 24 rows
S